---

# IT00CJ42: Search and Optimization Algorithms

**Restart the kernel and run all cells** before you turn this problem in, make sure everything runs as expected.

Make sure you fill in any place that says `YOUR CODE HERE`.

---

## Week 5

In [35]:
import math
import random

import numpy as np

### Solutions check
We use the function **check** to implement tests for your solution

In [36]:
def check(expression, message=""):
    if not expression:
        raise AssertionError(message)
    print(message + ": passed.")

### Task 1: Implement evaluate_objective

In [37]:
def evaluate_objective(permutation, distances, flow):
    total_cost = 0
    for i in range(len(permutation)):
        for j in range(len(permutation)):
            total_cost += flow[i][j] * distances[permutation[i]][permutation[j]]

    return total_cost

### Task 2: Simulated annealing

In [38]:
def simulated_annealing(distances, flow, temperature_max, temperature_min, cooling_rate):
    """
      Execute until the minmum temperature is reached
      Initialize a random input, and its corresponding value
      Take the delta between the current and new values
      If lower than zero or a random number between 0-1
      that is less than the metropolis acceptance value
      we get our new best input and corresponding value
      this is done to make it possible to also accept worse
      solutions to broaden the search
      Lower the temperature according to the cooling rate
      """
    current_permutation = list(range(len(distances)))
    random.shuffle(current_permutation)
    current_objective = evaluate_objective(current_permutation, distances, flow)

    best_permutation = current_permutation[:]
    best_objective = current_objective
    temperature = temperature_max
    iterations = 0

    while temperature > temperature_min:
        # Creates a new permutation by randomly swapping two elements in the current permutation. 
        new_permutation = current_permutation[:]
        i, j = random.sample(range(len(new_permutation)), 2)
        new_permutation[i], new_permutation[j] = new_permutation[j], new_permutation[i]

        new_objective = evaluate_objective(new_permutation, distances, flow)
        delta = new_objective - current_objective

        if delta < 0 or random.uniform(0, 1) < math.exp(-delta / temperature):
            current_permutation = new_permutation
            current_objective = new_objective

            if current_objective < best_objective:
                # Updates the best solution if the current solution has a better (lower) objective value.
                best_permutation = current_permutation[:]
                best_objective = current_objective
        # Lowers the temperature
        temperature *= cooling_rate
        iterations += 1

    return best_permutation, best_objective, iterations

#### Setting up the experiment

In [39]:
def load_dat(filename):
    # load in all rows into lists
    dat = [i.strip().split() for i in open(filename).readlines()]
    # remove empty lists
    res = [ele for ele in dat if ele != []]
    # convert strings to integers
    temp = [list(map(int, x)) for x in res]

    # first row: size, optimal value
    info = temp[0]
    size = info[0]

    # size*rows down is the distance
    distances = temp[1:size + 1]

    # size*rows after that is the flow
    flow = temp[size + 1:len(temp)]

    # return three seperate lists, (size, optimal value), (distance), (flow)
    return info, distances, flow

In [40]:
def run_one_algorithm(algorithm, distances, flows, trials=30, initial_seed=1):
    runs = []
    for i in range(trials):
        np.random.seed(initial_seed + i)
        random.seed(initial_seed + i)
        current = algorithm(distances, flows)
        print(f'{i + 1}: {current}')
        runs.append(current)
    best_run = min(runs, key=lambda t: t[1])
    return best_run

## Check solution: NUG5

In [41]:
info, D, F = load_dat("../res/data/nug5.dat")
optimal = info[1]
algorithm = (lambda distances, flows: simulated_annealing(distances, flows, 100, 0.05, 0.95))
best_run = run_one_algorithm(algorithm, D, F)
print(
    f'\nBest permutation: {best_run[0]}\nBest objective: {best_run[1]} \nOptimal objective: {info[1]}\n{best_run[2]} iterations')

1: ([2, 3, 4, 0, 1], 50, 149)
2: ([1, 3, 4, 0, 2], 50, 149)
3: ([1, 3, 4, 0, 2], 50, 149)
4: ([1, 3, 4, 0, 2], 50, 149)
5: ([1, 0, 2, 3, 4], 58, 149)
6: ([1, 3, 4, 0, 2], 50, 149)
7: ([3, 2, 4, 1, 0], 52, 149)
8: ([3, 1, 4, 2, 0], 52, 149)
9: ([1, 3, 4, 0, 2], 50, 149)
10: ([3, 1, 4, 2, 0], 52, 149)
11: ([2, 3, 4, 0, 1], 50, 149)
12: ([1, 3, 4, 0, 2], 50, 149)
13: ([1, 3, 4, 0, 2], 50, 149)
14: ([1, 3, 4, 0, 2], 50, 149)
15: ([2, 3, 4, 0, 1], 50, 149)
16: ([1, 3, 4, 0, 2], 50, 149)
17: ([1, 3, 4, 0, 2], 50, 149)
18: ([2, 3, 4, 0, 1], 50, 149)
19: ([1, 3, 4, 0, 2], 50, 149)
20: ([1, 3, 4, 0, 2], 50, 149)
21: ([1, 3, 4, 0, 2], 50, 149)
22: ([3, 2, 4, 1, 0], 52, 149)
23: ([3, 2, 4, 1, 0], 52, 149)
24: ([2, 3, 4, 0, 1], 50, 149)
25: ([3, 2, 4, 1, 0], 52, 149)
26: ([2, 3, 4, 0, 1], 50, 149)
27: ([1, 3, 4, 0, 2], 50, 149)
28: ([1, 0, 2, 3, 4], 58, 149)
29: ([2, 3, 4, 0, 1], 50, 149)
30: ([3, 2, 4, 1, 0], 52, 149)

Best permutation: [2, 3, 4, 0, 1]
Best objective: 50 
Optimal objective: 50
14

In [42]:
check(best_run[1] <= optimal * 1.10, "Best objective should be within 10% of optimal")

Best objective should be within 10% of optimal: passed.


## Check solution: NUG20

In [43]:
info, D, F = load_dat("../res/data/nug20.dat")
optimal = info[1]
algorithm = None
# Set the algorithm
algorithm = (lambda distance, flows: simulated_annealing(distance, flows, 1000, 0.001, 0.95))

best_run = run_one_algorithm(algorithm, D, F)
print(
    f'\nBest permutation: {best_run[0]}\nBest objective: {best_run[1]} \nOptimal objective: {info[1]}\n{best_run[2]} iterations')

1: ([2, 13, 8, 15, 12, 17, 3, 18, 19, 5, 9, 1, 14, 6, 0, 16, 10, 7, 11, 4], 2840, 270)
2: ([16, 18, 17, 9, 2, 0, 6, 1, 11, 13, 4, 3, 14, 7, 8, 5, 12, 19, 10, 15], 2744, 270)
3: ([9, 2, 18, 17, 13, 11, 14, 1, 10, 15, 0, 19, 6, 3, 8, 5, 7, 4, 16, 12], 2796, 270)
4: ([8, 12, 2, 5, 16, 15, 0, 9, 6, 4, 13, 18, 14, 1, 17, 10, 11, 7, 19, 3], 2788, 270)
5: ([12, 6, 4, 0, 16, 5, 19, 7, 11, 2, 3, 10, 1, 9, 17, 15, 18, 14, 13, 8], 2704, 270)
6: ([8, 15, 3, 13, 10, 9, 17, 1, 18, 11, 2, 4, 14, 19, 7, 5, 0, 12, 6, 16], 2810, 270)
7: ([16, 3, 18, 5, 12, 19, 6, 1, 4, 17, 7, 10, 14, 9, 8, 0, 11, 13, 15, 2], 2760, 270)
8: ([5, 19, 14, 9, 18, 12, 6, 3, 1, 17, 8, 11, 10, 15, 13, 2, 0, 7, 4, 16], 2786, 270)
9: ([8, 12, 5, 9, 4, 2, 13, 6, 11, 0, 17, 1, 19, 14, 16, 15, 3, 10, 7, 18], 2730, 270)
10: ([5, 12, 2, 9, 8, 16, 17, 4, 1, 13, 7, 19, 14, 3, 18, 0, 11, 6, 10, 15], 2772, 270)
11: ([16, 8, 3, 4, 12, 1, 18, 7, 19, 6, 13, 14, 10, 5, 11, 17, 15, 2, 9, 0], 2858, 270)
12: ([8, 2, 0, 15, 17, 9, 11, 10, 1, 13, 

In [44]:
check(best_run[1] <= optimal * 1.10, "Best objective should be within 10% of optimal")

Best objective should be within 10% of optimal: passed.


In [45]:
for i in range(30):
    np.random.seed(i)
    random.seed(i)
    info, D, F = load_dat("../res/data/nug20.dat")
    optimal = info[1]
    best_run = run_one_algorithm(algorithm, D, F)
    print(
        f'\nBest permutation: {best_run[0]}\nBest objective: {best_run[1]} \nOptimal objective: {info[1]}\n{best_run[2]} iterations')
    check(best_run[1] <= optimal * 1.10, "Best objective should be within 10% of optimal")

1: ([2, 13, 8, 15, 12, 17, 3, 18, 19, 5, 9, 1, 14, 6, 0, 16, 10, 7, 11, 4], 2840, 270)
2: ([16, 18, 17, 9, 2, 0, 6, 1, 11, 13, 4, 3, 14, 7, 8, 5, 12, 19, 10, 15], 2744, 270)
3: ([9, 2, 18, 17, 13, 11, 14, 1, 10, 15, 0, 19, 6, 3, 8, 5, 7, 4, 16, 12], 2796, 270)
4: ([8, 12, 2, 5, 16, 15, 0, 9, 6, 4, 13, 18, 14, 1, 17, 10, 11, 7, 19, 3], 2788, 270)
5: ([12, 6, 4, 0, 16, 5, 19, 7, 11, 2, 3, 10, 1, 9, 17, 15, 18, 14, 13, 8], 2704, 270)
6: ([8, 15, 3, 13, 10, 9, 17, 1, 18, 11, 2, 4, 14, 19, 7, 5, 0, 12, 6, 16], 2810, 270)
7: ([16, 3, 18, 5, 12, 19, 6, 1, 4, 17, 7, 10, 14, 9, 8, 0, 11, 13, 15, 2], 2760, 270)
8: ([5, 19, 14, 9, 18, 12, 6, 3, 1, 17, 8, 11, 10, 15, 13, 2, 0, 7, 4, 16], 2786, 270)
9: ([8, 12, 5, 9, 4, 2, 13, 6, 11, 0, 17, 1, 19, 14, 16, 15, 3, 10, 7, 18], 2730, 270)
10: ([5, 12, 2, 9, 8, 16, 17, 4, 1, 13, 7, 19, 14, 3, 18, 0, 11, 6, 10, 15], 2772, 270)
11: ([16, 8, 3, 4, 12, 1, 18, 7, 19, 6, 13, 14, 10, 5, 11, 17, 15, 2, 9, 0], 2858, 270)
12: ([8, 2, 0, 15, 17, 9, 11, 10, 1, 13, 